# Hierarchical MCMC for Polynomial Hyperelastic Model

## Data after Compression

In [ ]:
data = [-0.019397538181166286,
-0.03950739199689259,
-0.06038278777575503,
-0.08208157204829222,
-0.10466668288087849,
-0.12820667888715018,
-0.15277633280785963,
-0.17845729667032112,
-0.2053388451756016,
-0.23351870275250133]

## Initialize  FE

In [ ]:
from fenics import *
N=5
from numpy import where

mesh = UnitCubeMesh(N, N, N)
V = VectorFunctionSpace(mesh, "Lagrange", 1)
loadSteps = 10 
delta = 0.02
top =  CompiledSubDomain("near(x[2], side) && on_boundary", side = 1.0)
bottom = CompiledSubDomain("near(x[2], side) && on_boundary", side = 0.0)
c = Constant((0.0, 0.0, 0.0))
bc_bottom = DirichletBC(V, c, bottom)
vv = Function(V)
bc = DirichletBC(V, Constant((0.0, 0.0, 1.0)), top)
bc.apply(vv.vector())
top_dofs = where(vv.vector()==1.0)[0]

import numpy as np

param = np.array([0.1,0.2,0.01,0.01])

delta = np.linspace(0.02, 0.2,10)

I will first run an optimization in order to specify the priors knowledge.
The model below is aversion of MacroModel which is ready for the oprimazation

In [ ]:
def model(delta,C01,C10,C11,D1):

        ## Unwrap parameters

    #C01 = param[0]
    #C10 = param[1]
    #C11 = param[2]
    #D1 = param[3]

        # Define functions
    du = TrialFunction(V)            # Incremental displacement
    v  = TestFunction(V)             # Test function
    u  = Function(V)                 # Displacement from previous iteration

        # Kinematics
    d = u.geometric_dimension()
    I = Identity(d)             # Identity tensor
    F = I + grad(u)             # Deformation gradient
    C = F.T*F                   # Right Cauchy-Green tensor

        # Invariants of deformation tensors
    I1 = tr(C)
    J  = det(F)

    I2 = 0.5 * ( I1**2 - tr(C * C))

    barI1 = J**(-2./3.) * I1

    barI2 = J**(-4./3.) * I2

        # Stored strain energy density (compressible neo-Hookean model)
    psi = C10 * (barI1 - 3.)  + C01 * (barI2 - 3.) + C11 * (barI1 - 3.) * (barI2 - 3.) + D1 * (J - 1.)**2

        # Total potential energy
    Pi = psi * dx

        # Compute first variation of Pi (directional derivative about u in the direction of v)
    F = derivative(Pi, u, v)

        # Compute Jacobian of F
    J = derivative(F, u, du)

        # Loading loop
    Force=[]
    for i in range(0, len(delta)):
                    # Update non-homogeneous boundary condition for current load step
                r = Constant((0.0, 0.0, -delta[i]))
                bc_top = DirichletBC(V, r, top)
                bcs = [bc_bottom, bc_top]

                    # Solve variational problem
                solve(F == 0, u, bcs, J=J)

                    

                    # Output forces
                y = assemble(F)
                Force_top = 0
                for i in top_dofs:
                    Force_top += y[i]

                #print(Force_top)
                Force.append(Force_top)

    return np.array(Force)

Run the optimazation with starting points of the parameters [20,20,20,20]. 

In [ ]:
from scipy.optimize import curve_fit
popt,pcov = curve_fit(model,delta,data,p0=np.asarray([20,20,20,20]))

## Hierarchical MCMC for comperession

In [12]:
from fenics import *

from numpy import where

import numpy as np

import matplotlib.pyplot as plt
import random as rnd
###############################################################################
from smt.sampling_methods import LHS
from emukit.model_wrappers.gpy_quadrature_wrappers import BaseGaussianProcessGPy, RBFGPy
from emukit.quadrature.kernels.integration_measures import IsotropicGaussianMeasure
from emukit.quadrature.kernels import QuadratureRBFIsoGaussMeasure
from emukit.quadrature.methods.vanilla_bq import VanillaBayesianQuadrature
import GPy
import matplotlib.pyplot as plt
from scipy.stats import truncnorm
import math
from emukit.core.loop.user_function import UserFunctionWrapper
import random as rnd
import seaborn as sns
from typing import Tuple, List, Optional
from scipy.stats import norm
import _pickle as cPickle


ModuleNotFoundError: No module named 'Gen_data1'

# Macro Model for Compression

In [ ]:
class MacroModel():

    def __init__(self, N = 5):

        self.mesh = UnitCubeMesh(N, N, N)

        self.V = VectorFunctionSpace(self.mesh, "Lagrange", 1)
        
        self.loadSteps = 10   # cube [1,1,1] is pressed by 0.2 on top
        self.delta = 0.02
        self.N_MC = 30
        self.verbosity = True

        # Find the top and bottom of my domain
        self.top =  CompiledSubDomain("near(x[2], side) && on_boundary", side = 1.0)
        self.bottom = CompiledSubDomain("near(x[2], side) && on_boundary", side = 0.0)
        self.c = Constant((0.0, 0.0, 0.0))
        self.bc_bottom = DirichletBC(self.V, self.c, self.bottom) #boundary for bottom face = fixed

        # Find dofs on top face
        #ff = MeshFunction("size_t",mesh, mesh.topology().dim()-1, 0)
        #top.mark(ff, 1)
        self.vv = Function(self.V)
        self.bc = DirichletBC(self.V, Constant((0.0, 0.0, 1.0)), self.top)
        self.bc.apply(self.vv.vector())
        self.top_dofs = where(self.vv.vector()==1.0)[0]

    def setData(self, data, sigma_f):
        self.data = data
        self.sigma_f = sigma_f
        self.J = len(data)


    def apply(self, data, param, plotSolution = False, solutionFileName = "solution"):

        ## Unwrap parameters

        C01 = param[0]
        C10 = param[1]
        C11 = param[2]
        D1 = param[3]

        # Define functions
        du = TrialFunction(self.V)            # Incremental displacement
        v  = TestFunction(self.V)             # Test function
        u  = Function(self.V)                 # Displacement from previous iteration

        # Kinematics
        d = u.geometric_dimension()
        I = Identity(d)             # Identity tensor
        F = I + grad(u)             # Deformation gradient
        C = F.T*F                   # Right Cauchy-Green tensor

        # Invariants of deformation tensors
        I1 = tr(C)
        J  = det(F)

        I2 = 0.5 * ( I1**2 - tr(C * C))

        barI1 = J**(-2./3.) * I1

        barI2 = J**(-4./3.) * I2

        # Stored strain energy density (compressible neo-Hookean model)
        psi = C10 * (barI1 - 3.)  + C01 * (barI2 - 3.) + C11 * (barI1 - 3.) * (barI2 - 3.) + D1 * (J - 1.)**2

        # Total potential energy
        Pi = psi * dx

        # Compute first variation of Pi (directional derivative about u in the direction of v)
        F = derivative(Pi, u, v)

        # Compute Jacobian of F
        J = derivative(F, u, du)

        # Loading loop
        Force=[]
        for i in range(1, self.loadSteps + 1):

                    if(self.verbosity):
                        print("Load step - " + str(i))

                    # Update non-homogeneous boundary condition for current load step
                    r = Constant((0.0, 0.0, -i * self.delta))
                    bc_top = DirichletBC(self.V, r, self.top)
                    bcs = [self.bc_bottom, bc_top]

                    # Solve variational problem
                    solve(F == 0, u, bcs, J=J)

                    # Save solution in VTK format
                    if(plotSolution):
                        file = File(solutionFileName + str(i) + ".pvd");
                        file << u;

                    # Output forces
                    y = assemble(F)
                    Force_top = 0
                    for i in self.top_dofs:
                        Force_top += y[i]

                    print(Force_top)
                    Force.append(Force_top)

        #
        assert len(Force) == len(data), "Force list not same length as data list"

        #return -np.sum((np.asarray(Force) - np.asarray(self.data))**2)/ (2.0 * self.sigma_f**2)
        Phi = -np.sum((np.asarray(Force) - np.asarray(data))**2)/ (2.0 * self.sigma_f**2)
        #return np.exp(Phi)
        return Phi
        
    def evaluateDensity(self, misfit):
        return -0.5 * np.sum(misfit)**2 / (self.sigmaf**2)
##############################################################################################################
    def press_fem(self) -> Tuple[UserFunctionWrapper, List[Tuple[float, float]]]:
        
        integral_bound = [(1,15),(0.2,0.4)]
        return UserFunctionWrapper(self._press_fem), integral_bound

    def _press_fem(self, data,param:np.ndarray) -> np.ndarray:
        Likelihood=self.apply(data, param)
        return np.reshape(np.array(Likelihood),(-1,1))
##############################################################################################################
    def singleGP(self, data):
        #########################################
        xlimits = np.array([[0,0.2], [0,0.3],[0,0.2],[0,0.3]])   #for polynomial Mooney Rivlin
        sampling = LHS(xlimits=xlimits)

        train_number =300
        theta = sampling(train_number)        
        ########################################
        
        self.X_init=np.zeros((train_number,4))
        Y=[]
        for i in range(0,train_number):                #initial points to GP 
                function_input=theta[i]
                self.X_init[i]=theta[i]
                Y_= np.reshape(user_function.f(data, function_input),(-1,1))
                Y.append(Y_)
            
        self.Y_init=np.reshape(np.array(Y),(-1,1))

        gpy_model = GPy.models.GPRegression(X=self.X_init, Y=self.Y_init,kernel=GPy.kern.Matern52(input_dim=self.X_init.shape[1], ARD=False))  
       
        gpy_model.optimize(messages=True)
        gpy_model.optimize_restarts(num_restarts = 1)

        return gpy_model
    
  
    def trainAll(self):
        
        self.GPs=[]
        
        for j in range(0,self.J):
            GP=self.singleGP(self.data[j])
            self.GPs.append(GP)
        return self.GPs  

#################################################################################################################
    def theta_given_phi(self, phi, theta):
        return norm.logpdf(theta[0], loc=phi[0], scale=phi[1]) + norm.logpdf(theta[1], loc=phi[2], scale=phi[3]) + norm.logpdf(theta[2], loc=phi[4], scale=phi[5]) + norm.logpdf(theta[3], loc=phi[6], scale=phi[7])

    def logLike(self,phi):
        ll = 0.0
        for j in range(self.J): # For each experiment
            tmp = 0.0
            for i in range(self.N_MC):
                theta =np.array( [np.random.normal(phi[0],phi[1]),np.random.normal(phi[2],phi[3]),np.random.normal(phi[4],phi[5]),np.random.normal(phi[6],phi[7])]) 
                gp_theta=np.reshape(theta,(1,-1))
                part1 =self.GPs[j].predict(gp_theta) 
                part2  = self.theta_given_phi(phi, theta)
                #tmp+=float(part1[0])  *part2                        
                tmp+=float(part1[0])  + np.log(part2)
            if tmp<=0:
                tmp=0.01
            #ll += np.log(tmp/self.N_MC)
            ll += tmp/self.N_MC
        return ll             


# set up the model

In [ ]:
loadSteps = 10
delta = 0.02
saveSolution = True
verbosity = True

Data_compression = cPickle.load( open( "CompressionData.p", "rb" ) )
myModel = MacroModel()
myModel.setData(Data_compression,0.1)
user_function, integral_bounds = myModel.press_fem()
tr=myModel.trainAll()


start MCMC

In [ ]:
ndraws = 3000  # number of draws from the distribution
phi=np.zeros((ndraws,8))
#theta=np.zeros((1,2))
prop_phi=np.zeros((ndraws,8))
sigma1_sq=np.zeros((ndraws,1))
sigma2_sq=np.zeros((ndraws,1))
a=[1]
sigma1_sq[0]=1
sigma1_sq_it=1
k=0
ni=0
m=1.1
a_star=0.60
acceptanceCount=0
phi[0]=[1,1,1,1,1,1,1,1]


In [ ]:
logLikelihood = myModel.logLike(phi[0])

for it in range(1,ndraws):
    prop_phi[it][0]=np.sqrt(1-sigma1_sq_it**2)*phi[it-1][0] + 0.1*sigma1_sq_it*float(truncnorm.rvs(-1,1,size=1))    
    prop_phi[it][1]=np.sqrt(1-sigma1_sq_it**2)*phi[it-1][1] + 0.01*sigma1_sq_it*float(truncnorm.rvs(-1,1,size=1))
    prop_phi[it][2]=np.sqrt(1-sigma1_sq_it**2)*phi[it-1][2] + 0.2*sigma1_sq_it*float(truncnorm.rvs(-1,1,size=1))
    prop_phi[it][3]=np.sqrt(1-sigma1_sq_it**2)*phi[it-1][3] + 0.01*sigma1_sq_it*float(truncnorm.rvs(-1,1,size=1))
    prop_phi[it][4]=np.sqrt(1-sigma1_sq_it**2)*phi[it-1][4] + 0.09*sigma1_sq_it*float(truncnorm.rvs(-1,1,size=1))    
    prop_phi[it][5]=np.sqrt(1-sigma1_sq_it**2)*phi[it-1][5] + 0.01*sigma1_sq_it*float(truncnorm.rvs(-1,1,size=1))
    prop_phi[it][6]=np.sqrt(1-sigma1_sq_it**2)*phi[it-1][6] + 0.1*sigma1_sq_it*float(truncnorm.rvs(-1,1,size=1))
    prop_phi[it][7]=np.sqrt(1-sigma1_sq_it**2)*phi[it-1][7] + 0.01*sigma1_sq_it*float(truncnorm.rvs(-1,1,size=1))
    
    
    prop_phi[it]=abs(prop_phi[it])
    
    #############################    
#update the adaptation
    if it>100*(ni+1):
        k=k+1
        ni=ni+1
        #update of sigma_sq
        log_sigma1_sq=np.log(sigma1_sq[k-1]) + m**(-k)*(np.mean(np.array([a]))-a_star)
        sigma1_sq[k]=np.exp(log_sigma1_sq)
        sigma1_sq_it=float(sigma1_sq[k]) 

    ##############################
    prop_logLikelihood = myModel.logLike(prop_phi[it])


    alpha= prop_logLikelihood -logLikelihood
    alpha = min(0,alpha)
            
    
    if     np.log(rnd.random())<alpha:
    
           acceptanceCount=acceptanceCount+1
           phi[it]=prop_phi[it]
           logLikelihood=prop_logLikelihood
           a=a+[1]
    else:
           phi[it]=phi[it-1]
           a=a+[0]
           
